In [162]:
import sys
sys.path.append('../')

In [163]:
import tensorflow as tf
from RCWA.Domain import Domain
from RCWA.Modes import Modes
from RCWA.EigenMode import EigenMode
from RCWA.ScatterMat import ScatterMatBuilder
from RCWA.Device import SlantGrating
from RCWA import Utils
import numpy as np
import matplotlib.pyplot as plt
import imageio.v3 as iio
import io
from PIL import Image
from pygifsicle import optimize

In [164]:


# Reference:
# [1] M. A. Golub and A. A. Friesem,
# “Effective grating theory for resonance domain surface-relief diffraction gratings,”
# J. Opt. Soc. Am. A, vol. 22, no. 6, p. 1115, Jun. 2005, doi: 10.1364/JOSAA.22.001115.

period = 0.3276

n1 = 1
n2 = 2

er1 = n1**2
er2 = n2**2
ur1 = 1
ur2 = 1

AOI = 10

domain = Domain()
domain.set_period_centered(period, period)

modes = Modes(domain)
modes.set_harmonics(10, 0)


modes.set_incidence_AOI_POI(
    AOI=np.deg2rad(AOI),
    POI=np.deg2rad(0))

modes.set_wavelength(0.2)

# Default Matrix
sbuilder = ScatterMatBuilder(modes)
ref_mode = EigenMode(modes)
trn_mode = EigenMode(modes)
ref_mode.from_homogeneous(er1, ur1)
trn_mode.from_homogeneous(er2, ur2)
Sref = sbuilder.BuildScatterRef(ref_mode)
Strn = sbuilder.BuildScatterTrn(trn_mode)

Sglobal = Sref

# Create Device

x, y = domain.get_coordinate(modes.num_modes_x*20, modes.num_modes_y*4)

mask = (x>-0.1036) * (x<0.0721)
er = (n2**2 - n1**2) * mask + n1**2
eigenmode = EigenMode(modes)
eigenmode.from_material_er(er)
S = sbuilder.BuildScatter(eigenmode, 1)


Sglobal = Sglobal @ S
Sglobal = Sglobal @ Strn

# Incidence
delta = (modes.mx == 0)*(modes.my == 0)

pol_angle_deg = 90

# Set amplitudes for s and p polarizations
pol_angle = np.deg2rad(pol_angle_deg)

amp_s = tf.sin(pol_angle)  # 90 degree, TE
amp_p = tf.cos(pol_angle)  # 0 degree, TM

# Calculate the polarization vector based on s and p amplitudes
pol = modes.pol_vec_p * amp_p + modes.pol_vec_s * amp_s

# Calculate the incident electric field components
Einc = tf.cast(
    tf.concat([delta*pol[0], delta*pol[1]], 0), tf.dtypes.complex128)[:, None]
Einc_z = delta*pol[2]

# Calculate the incident intensity
Iinc = tf.reduce_sum(np.abs(Einc)**2)+tf.reduce_sum(np.abs(Einc_z)**2)

# Calculate the longitudinal wave vector components
kz_r = tf.sqrt((n1**2-modes.kx**2-modes.ky**2).astype('complex'))
kz_t = tf.sqrt((n2**2-modes.kx**2-modes.ky**2).astype('complex'))

# Calculate the electric field components using the scattering matrix
Cref = Sglobal.value[0]@Einc
Ctrn = Sglobal.value[2]@Einc

Eref = tf.reshape(Cref, [2, -1])
Etrn = tf.reshape(Ctrn, [2, -1])

# Calculate the longitudinal electric field components
Eref_z = -(Eref[0]*modes.kx+Eref[1]*modes.ky)/kz_r
Etrn_z = -(Etrn[0]*modes.kx+Etrn[1]*modes.ky)/kz_t

# Calculate the reflected and transmitted intensities
Iref = tf.reduce_sum(tf.abs(Eref)**2, 0)+tf.abs(Eref_z)**2
Itrn = tf.reduce_sum(tf.abs(Etrn)**2, 0)+tf.abs(Etrn_z)**2

# Calculate the reflection and transmission coefficients
R = Iref*tf.math.real(kz_r)/modes.k0z/Iinc
T = Itrn*tf.math.real(kz_t)/modes.k0z/Iinc


print(np.sum(R+T))
print("Error:", 1-np.sum(R+T))


0.9999999999990695
Error: 9.304779169383437e-13


In [165]:
Eref.shape

TensorShape([2, 21])

In [166]:
Einc

<tf.Tensor: shape=(42, 1), dtype=complex128, numpy=
array([[0.00000000e+00+0.j],
       [0.00000000e+00+0.j],
       [0.00000000e+00+0.j],
       [0.00000000e+00+0.j],
       [0.00000000e+00+0.j],
       [0.00000000e+00+0.j],
       [0.00000000e+00+0.j],
       [0.00000000e+00+0.j],
       [0.00000000e+00+0.j],
       [0.00000000e+00+0.j],
       [6.03020831e-17+0.j],
       [0.00000000e+00+0.j],
       [0.00000000e+00+0.j],
       [0.00000000e+00+0.j],
       [0.00000000e+00+0.j],
       [0.00000000e+00+0.j],
       [0.00000000e+00+0.j],
       [0.00000000e+00+0.j],
       [0.00000000e+00+0.j],
       [0.00000000e+00+0.j],
       [0.00000000e+00+0.j],
       [0.00000000e+00+0.j],
       [0.00000000e+00+0.j],
       [0.00000000e+00+0.j],
       [0.00000000e+00+0.j],
       [0.00000000e+00+0.j],
       [0.00000000e+00+0.j],
       [0.00000000e+00+0.j],
       [0.00000000e+00+0.j],
       [0.00000000e+00+0.j],
       [0.00000000e+00+0.j],
       [1.00000000e+00+0.j],
       [0.00000000e+

In [167]:
np.reshape(Einc, [2, -1])[1]

array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j])

In [168]:
modes.k0

31.41592653589793

In [173]:
C = np.reshape(Einc+Cref, [2, -1])[1]
C = np.reshape(C, [modes.num_modes_y, modes.num_modes_x])
C = np.pad(C, [[0], [100]])
x, y = domain.get_coordinate(C.shape[1], C.shape[0])
E = np.fft.ifft2(np.fft.ifftshift(C)) * np.exp(1j * modes.k0 * modes.k0x * x)
print(E.shape)
N = 20

for i in range(N):
    fig = plt.figure(figsize=(5, 1), frameon=False)
    plt.clf()
    ax = fig.add_axes([0, 0, 1, 1])
    ax.axis('off')
    a = i/N*2*np.pi
    ax.plot(np.real(E[0]*np.exp(1j*a)), c='#FF9800')
    plt.ylim([-0.01, 0.01])
    plt.savefig(f"./output1/{i:04d}.png", format='png')
    plt.close()



(1, 221)


In [170]:
np.array(frames).shape

(10, 480, 640, 4)

In [174]:
C = np.reshape(Etrn, [2, -1])[1]
C = np.reshape(C, [modes.num_modes_y, modes.num_modes_x])
C = np.pad(C, [[0], [100]])
x, y = domain.get_coordinate(C.shape[1], C.shape[0])
E = np.fft.ifft2(np.fft.ifftshift(C)) * np.exp(1j * modes.k0 * modes.k0x * x)
print(E.shape)

N = 20

for i in range(N):
    fig = plt.figure(figsize=(5, 1), frameon=False)
    plt.clf()
    ax = fig.add_axes([0, 0, 1, 1])
    ax.axis('off')
    a = i/N*2*np.pi
    ax.plot(np.real(E[0]*np.exp(1j*a)), c='#FF9800')
    plt.ylim([-0.01, 0.01])
    plt.savefig(f"./output2/{i:04d}.png", format='png')
    plt.close()

(1, 221)


In [172]:
T

<tf.Tensor: shape=(21,), dtype=float64, numpy=
array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.06580447, 0.06864765, 0.25706559,
       0.16495806, 0.34898613, 0.01141329, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])>